In [ ]:
%run ./kitaev_ansatz.ipynb

## Optimize using any Scipy optimizer (e.g. Cobyla), and vary shots and depth

In [ ]:
num_qubits = 4
depth = 4
shots = 8192
minimize(cost_function, create_random_param(depth = depth), args=(num_qubits, shots), method='cobyla')

## Optimize using any Qiskit's built-in optimizer, which allows for statevector or shot-based optimization

In [ ]:
from qiskit.opflow import I, X, Y, Z

hamiltonians = {'4': (I^X^X^I) + (X^I^I^X) + (Y^Y^I^I) + (I^I^Y^Y) + (Z^I^Z^I) + (I^Z^I^Z),
                '8': (I^I^I^Z^I^I^Z^I) + (Z^I^I^I^I^Z^I^I) + (I^I^I^I^Z^I^I^Z) + (I^Z^Z^I^I^I^I^I) + (I^I^Y^Y^I^I^I^I) + (I^I^I^I^I^I^Y^Y) + (Y^I^I^I^Y^I^I^I) + (I^Y^I^I^I^Y^I^I) + (I^I^I^X^I^I^I^X) + (I^I^X^I^I^I^X^I) + (X^X^I^I^I^I^I^I) + (I^I^I^I^X^X^I^I),
                '16': \
                    (I^I^I^I^I^I^I^I^I^I^I^I^I^I^X^X) + \
                    (I^I^I^I^I^I^X^X^I^I^I^I^I^I^I^I) + \
                    (X^I^I^I^I^I^I^I^X^I^I^I^I^I^I^I) + \
                    (I^X^I^I^I^I^I^I^I^X^I^I^I^I^I^I) + \
                    (I^I^I^I^I^X^I^I^I^I^I^I^I^X^I^I) + \
                    (I^I^I^I^X^I^I^I^I^I^I^I^X^I^I^I) + \
                    (I^I^X^X^I^I^I^I^I^I^I^I^I^I^I^I) + \
                    (I^I^I^I^I^I^I^I^I^I^X^X^I^I^I^I) + \
                    (I^I^I^I^I^I^I^Y^I^I^I^I^I^I^I^Y) + \
                    (I^I^I^I^I^I^Y^I^I^I^I^I^I^I^Y^I) + \
                    (I^I^I^I^I^I^I^I^Y^Y^I^I^I^I^I^I) + \
                    (Y^Y^I^I^I^I^I^I^I^I^I^I^I^I^I^I) + \
                    (I^I^I^I^Y^Y^I^I^I^I^I^I^I^I^I^I) + \
                    (I^I^Y^I^I^I^I^I^I^I^I^I^I^Y^I^I) + \
                    (I^I^Y^I^I^I^I^I^I^I^Y^I^I^I^I^I) + \
                    (I^I^I^Y^I^I^I^I^I^I^I^Y^I^I^I^I) + \
                    (I^I^I^I^I^I^I^I^I^I^I^I^Z^I^I^Z) + \
                    (I^I^I^I^I^Z^Z^I^I^I^I^I^I^I^I^I) + \
                    (I^I^I^I^I^I^I^I^Z^I^I^Z^I^I^I^I) + \
                    (I^Z^Z^I^I^I^I^I^I^I^I^I^I^I^I^I) + \
                    (Z^I^I^I^I^I^Z^I^I^I^I^I^I^I^I^I) + \
                    (I^I^I^I^I^I^I^I^I^Z^I^I^I^I^Z^I) + \
                    (I^I^I^Z^Z^I^I^I^I^I^I^I^I^I^I^I) + \
                    (I^I^I^I^I^I^I^I^I^I^Z^I^I^Z^I^I)
               }

In [ ]:
NUM_QUBITS = 8
DEPTH = 8
SHOTS = 8192

seed = algorithm_globals.random_seed

quant_inst = QuantumInstance(Aer.get_backend('statevector_simulator'), seed_transpiler = seed, seed_simulator = seed)
# quant_inst = QuantumInstance(Aer.get_backend('qasm_simulator', shots = SHOTS), seed_transpiler = seed, seed_simulator = seed)

optimizer = COBYLA(maxiter = 20000)
# optimizer = ADAM(maxiter = 20000,tol = 0.0001, lr = 0.001)

initial = create_random_param(depth = DEPTH)
ansatz = create_circuit(num_qubits = NUM_QUBITS, params = ParameterVector('p',DEPTH*6,))
vqe = VQE(ansatz = ansatz, optimizer = optimizer, initial_point = initial, quantum_instance = quant_inst)
result = vqe.compute_minimum_eigenvalue(operator = hamiltonians[str(NUM_QUBITS)])
print(result)

## Optimize using sequential minimal optimization 
https://journals.aps.org/prresearch/pdf/10.1103/PhysRevResearch.2.043158

In [ ]:
"""Find the energy of the shifted parameter using our cost function"""
def find_shifted_energy(index, shift, params):
    params_clone = copy.deepcopy(params)
    params_clone[index] = params[index] + shift
    shifted_energy = cost_function(params_clone, NUM_QUBITS, SHOTS)
    return shifted_energy

def find_coefficient_vector(params, index, energy):
    S = 8 #how many times this parameter appears in the circuit
    angle_shifts = (2 * np.pi / (2*S+1)) * np.arange(2*S+1) #set of angles from 0 to 2*pi to shift by 2*S+1 cuts
    shifted_energies = np.zeros(2*S+1)
    shifted_energies[0] = energy
    for i in range(1, len(angle_shifts)):
        shifted_energies[i] = find_shifted_energy(index, angle_shifts[i], params)  
    coefficient_vector = np.fft.rfft(shifted_energies)
    return coefficient_vector                

def single_cost_function(parameter, coeffs):          
    energy = float(coeffs[0])
    for i in range(len(coeffs) - 1):
        energy += 2 * np.real(coeffs[i+1] * np.exp(1j * (i+1) * parameter))
    energy /= 2 * len(coeffs) - 1
    return float(energy)

def update(params, index, coeffs):
    result = minimize(single_cost_function, 0, args = (coeffs), method = 'cobyla')
    params[index] = params[index] + result['x']
    updated_energy = result['fun']
    return params, updated_energy

In [ ]:
from time import time
import copy

NUM_QUBITS = 4
DEPTH = 4
MAX_RUNS = 3
SHOTS = 16386

energy_array = []
num_runs = 0
params=np.zeros(DEPTH*6)

start_time = time()
for num_runs in range(MAX_RUNS):
    print('\033[1m' + 'On loop ' + str(num_runs+1) + ' of ' + str(MAX_RUNS) + '\033[1m' + '\n')
    
    if num_runs == 0:
        previous_energy = cost_function(params, NUM_QUBITS, SHOTS)
        energy_array.append(previous_energy)
    else:
        previous_energy = energy_array[-1]
    
    for i in tqdm(range(len(params))):
        if i == 0:
            energy = previous_energy
        index = i
        coeffs = find_coefficient_vector(params, index, energy)
        params, energy = update(params, index, coeffs)
        energy_array.append(energy)
        print("energy: ", cost_function(params, NUM_QUBITS, SHOTS))
        print("params: ", params)
        print("Time elapsed: ", time() - start_time)
        print('\n')

## Optimize using CMA-ES
https://arxiv.org/pdf/1604.00772.pdf \
https://github.com/CMA-ES/pycma

In [ ]:
import cma

NUM_QUBITS = 4
SHOTS = 16384
es = cma.CMAEvolutionStrategy(create_random_param(depth = 4), 0.5)
es.optimize(cost_function, args = (NUM_QUBITS, SHOTS))

## Measure expressivity by plotting the entanglement spectrum

In [ ]:
"""Here we find the entanglement spectrum"""

find_entanglement_spectrum = True

if find_entanglement_spectrum == True:
    
    def schmidt_spectrum(state):

        matrix = np.array(state).reshape(int(np.sqrt(len(state))), int(np.sqrt(len(state))))
        u, vector, v = np.linalg.svd(matrix)
        return vector

    NUM_QUBITS = 16
    ITERATIONS = 100
    depth_range = range(1, NUM_QUBITS + 1)    

    #"""""""""""""""""""""" Graph config """""""""""""""""""#
    colors = plt.cm.viridis(np.linspace(0, 1, NUM_QUBITS + 1))
    fig, ax = plt.subplots(dpi = 100)
    ax.set_xlabel(r'$k$')
    ax.set_ylabel(r'$\xi_k$')
    norm = mpl.colors.BoundaryNorm(list(range(num_qubits + 1)), plt.cm.viridis.N)
    scalar_mappable = plt.cm.ScalarMappable(norm = norm, cmap = plt.cm.viridis)
    clb = fig.colorbar(scalar_mappable)
    clb.ax.set_title(r'$p$')
    #""""""""""""""""""""""""""""""""""""""""""""""""""""""#


    for depth in tqdm(depth_range):
        all_es = []
        for _ in range(ITERATIONS):
            random_param = create_random_param(depth)
            state = create_state_vector(NUM_QUBITS, random_param)
            schmidt_coeffs = schmidt_spectrum(state)
            all_es.append(np.log(schmidt_coeffs))
        mean_es = np.array(all_es, dtype = object).mean(axis = 0)
        ax.plot(mean_es, linestyle = 'solid', markersize = 2, color = colors[depth])

    all_haar = []
    for i in tqdm(range(ITERATIONS)):
        state = random_statevector(2 ** NUM_QUBITS).data
        schmidt_coefficients=schmidt_spectrum(state)
        spectrum = np.log(schmidt_coefficients)
        all_haar.append(spectrum)
    mean_haar = np.array(all_haar, dtype=object).mean(axis = 0)
    ax.plot(mean_haar, linestyle = 'dotted', color = '#000000', markersize = 2)

    plt.show()

## Investigate barren plateaus by plotting the variance of the gradient of the cost function

In [ ]:
hamiltonian4 = \
'ZIZI \
IZIZ \
XIIX \
IXXI \
YYII \
IIYY'

hamiltonian8 = \
'IIIZIIZI \
ZIIIIZII \
IIIIZIIZ \
IZZIIIII \
IIYYIIII \
IIIIIIYY \
YIIIYIII \
IYIIIYII \
IIIXIIIX \
IIXIIIXI \
XXIIIIII \
IIIIXXII'

hamiltonian12 = \
'IIIZZIIIIIII \
IIIIIIIIZIIZ \
IZZIIIIIIIII \
IIIIIIZIIZII \
YIIIIIYIIIII \
IYIIIIIYIIII \
IIYYIIIIIIII \
IIIIYIIIIIYI \
IIIIIYIIIIIY \
IIIIIIIIYYII \
XXIIIIIIIIII \
IIIIIIXXIIII \
IIXIIIIIXIII \
IIIXIIIIIXII \
IIIIXXIIIIII \
IIIIIIIIIIXX'

hamiltonian16 = \
'IIIIIIIIIIIIIIXX \
IIIIIIXXIIIIIIII \
XIIIIIIIXIIIIIII \
IXIIIIIIIXIIIIII \
IIIIIXIIIIIIIXII \
IIIIXIIIIIIIXIII \
IIXXIIIIIIIIIIII \
IIIIIIIIIIXXIIII \
IIIIIIIYIIIIIIIY \
IIIIIIYIIIIIIIYI \
IIIIIIIIYYIIIIII \
YYIIIIIIIIIIIIII \
IIIIYYIIIIIIIIII \
IIYIIIIIIIIIIYII \
IIYIIIIIIIYIIIII \
IIIYIIIIIIIYIIII \
IIIIIIIIIIIIZIIZ \
IIIIIZZIIIIIIIII \
IIIIIIIIZIIZIIII \
IZZIIIIIIIIIIIII \
ZIIIIIZIIIIIIIII \
IIIIIIIIIZIIIIZI \
IIIZZIIIIIIIIIII \
IIIIIIIIIIZIIZII'

In [ ]:
I = [[1,0],[0,1]]
X = [[0,1],[1,0]]
Y = [[0,-1j],[1j,0]]
Z = [[1,0],[0,-1]]

def op_to_matrix(op):
    if op == 'I':
        return [[1,0],[0,1]]
    elif op == 'X':
        return [[0,1],[1,0]]
    elif op == 'Y':
        return [[0,-1j],[1j,0]]
    elif op == 'Z':
        return [[1,0],[0,-1]]
    
def string_to_matrix(string):
    matrix = np.array(op_to_matrix(string[0]), dtype='complex64')
    for i in range(1,len(string)):
        matrix = scipy.kron(matrix, np.array(op_to_matrix(string[i]), dtype='complex64'))
    return matrix

def ham_to_matrix(qubits):
    hamiltonian = []
    if qubits == 4:
        hamiltonian = hamiltonian4
    elif qubits == 8:
        hamiltonian = hamiltonian8
    elif qubits == 12:
        hamiltonian = hamiltonian12
    elif qubits == 16:
        hamiltonian = hamiltonian16
    h_matrix = np.zeros((2 ** qubits, 2 ** qubits), dtype='complex64')
    for i in range(0,len(hamiltonian),qubits + 1):
        string = hamiltonian[i:i + qubits]
        h_matrix += string_to_matrix(string)
    return h_matrix

In [ ]:
hamiltonians = []
hamiltonians.append(ham_to_matrix(4))
hamiltonians.append(ham_to_matrix(8))
hamiltonians.append(ham_to_matrix(12))
hamiltonians.append(ham_to_matrix(16))
print('Hamiltonians calculated.')

# recommended to save this 16 qubit Hamiltonian and then load it to save time
# np.save('hamiltonian16',hamiltonians[3])
# hamiltonians.append(np.load('~/hamiltonian16.npy'))

In [ ]:
def gradient(cost1, cost2, delta):
    return (cost2 - cost1) / delta

DELTA = 1e-6
variances = []
QUBIT_RANGE = [4, 8, 12, 16]

for num_qubits in tqdm(QUBIT_RANGE):
    derivatives = []
    layers = 3 * num_qubits # this should be some positive natural number
    param_to_change = layers // 2 + 1
    hamiltonian = hamiltonians[num_qubits // 4 - 1]
    NUM_SETS = 100
    if num_qubits == 16: #must be smaller as this operation is lengthy
        NUM_SETS = 30
    for i in tqdm(range(NUM_SETS)):
        params1 = create_random_param(layers)
        params2 = copy.deepcopy(params1)
        params2[param_to_change] += DELTA

        psi1 = create_unitary(params1, num_qubits)
        psi2 = create_unitary(params2, num_qubits)
        cost1 = cost(psi1, hamiltonian, num_qubits)
        cost2 = cost(psi2, hamiltonian, num_qubits)

        derivatives.append(gradient(cost1, cost2, DELTA))
        
    variances.append(np.var(derivatives))

In [ ]:
# recommended to save the variances for various depths, load them later, and plot them all at once. 
# e.g.
# np.save('depth_x_variances',variances)
# depth1n_variances = np.load('/home/abhuynh/kitaev-square-octagon-model/src/depth1n_variances.npy')

plt.xticks(QUBIT_RANGE)
plt.ylabel('Var' + r'$_{\theta}[\partial C({\bf \theta})]$', fontsize=18)
plt.xlabel(r'$n$', fontsize = 18)
plt.semilogy(QUBIT_RANGE, variances, linestyle = '-', marker = 'o', color = 'blue', label = 'p = x')
plt.legend()
plt.show()